# Librerías

In [1]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import math

from scipy import stats
from scipy.stats import shapiro
from scipy.stats import levene
from scipy.stats import norm
from scipy.stats import mannwhitneyu
from scipy.stats import skew

In [2]:
df= pd.read_csv(r'Datasets\draft_picks.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets\\draft_picks.csv'

# AB Testing

## Configuración previa

In [ ]:
#Nos quedamos con métricas por partido para una mejor comparación y el top 20 de los picks
columns_of_interest = ['PTS per game', 'TBR per game', 'AST per game', 'WS', 'Pick','College']
df_AB = df[columns_of_interest]
top_20_pick = df_AB[df_AB['Pick'].between(1, 20)]

#Hacemos distinción entre jugadores drafteados de Colleges versus resto mundo
top_20_pick['College'] = top_20_pick['College'].apply(lambda x: 'US' if x != 'Out US' else x)

C:\Users\sfeli\AppData\Local\Temp\ipykernel_13144\1704981659.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_20_pick['College'] = top_20_pick['College'].apply(lambda x: 'US' if x != 'Out US' else x)


## Planteamiento de la hipótesis

Hipótesis nula (H0): No hay diferencia en las métricas de rendimiento entre los jugadores que han cursado en universidades americanas y del resto de fuera de USA.

Hipótesis alternativa (H1): Los jugadores de las universidades de USA tienen mejores métricas de rendimiento que el resto.

Este test se realiza con los jugadores drafteados en el top 20, ya que deberían ser los mejores de su promoción. Se realiza con métricas por partido para una mejor comparación.

## Creación de los Grupos


In [ ]:
#Filtramos los grupos
us_players = top_20_pick[top_20_pick['College'] == 'US']
out_us_players = top_20_pick[top_20_pick['College'] == 'Out US']

## Tamaño de la muestra

In [ ]:
#Definición de la función para calcular el tamaño de la muestra
def calcular_tamano_muestra(alpha, beta, d):
    Z_alpha_over_2 = norm.ppf(1 - alpha / 2)
    Z_beta = norm.ppf(1 - beta)
    n = ((Z_alpha_over_2 + Z_beta)**2) * 2 / d**2
    return math.ceil(n)

#Parámetros del cálculo
alpha = 0.05
beta = 0.2
d = 0.5

#Cálculo del tamaño de muestra necesario
tamaño_muestra_necesario = calcular_tamano_muestra(alpha, beta, d)
print(f'Tamaño de muestra necesario por grupo: {tamaño_muestra_necesario}')


tamaño_muestra_actual_us = len(us_players)
tamaño_muestra_actual_out_us = len(out_us_players)

print(f'Tamaño actual de la muestra - US players: {tamaño_muestra_actual_us}')
print(f'Tamaño actual de la muestra - Out US players: {tamaño_muestra_actual_out_us}')

#Comparamos el tamaño de muestra necesario con el de nuestra muestra
if tamaño_muestra_actual_us >= tamaño_muestra_necesario and tamaño_muestra_actual_out_us >= tamaño_muestra_necesario:
    print("El tamaño de la muestra es suficiente para el análisis.")
else:
    print("El tamaño de la muestra no es suficiente para el análisis.")

Tamaño de muestra necesario por grupo: 63
Tamaño actual de la muestra - US players: 1269
Tamaño actual de la muestra - Out US players: 99
El tamaño de la muestra es suficiente para el análisis.


## Prueba de Levene

In [ ]:
#Prueba de Levene para evaluar la igualdad de las varianzas (Se excluye 'Pick' y 'College')

columns_of_interest = ['WS', 'PTS per game', 'TBR per game', 'AST per game']

for column in columns_of_interest:
    stat, p = levene(us_players[column], out_us_players[column])
    print(f'Levene test - {column}: Levene stat = {stat}, p-value = {p}')

Levene test - WS: Levene stat = 1.8976381917588712, p-value = 0.1685685685877363
Levene test - PTS per game: Levene stat = 1.5875577347236287, p-value = 0.20789062421643306
Levene test - TBR per game: Levene stat = 0.09289391722478856, p-value = 0.7605763090620872
Levene test - AST per game: Levene stat = 0.0024496108894978257, p-value = 0.9605332176072402


En todos los casos, el p-valor de la prueba de Levene es mayor que el nivel de significancia (0.05). Esto indica que no hay evidencia suficiente para afirmar que las varianzas son significativamente diferentes entre los jugadores de universidades americanas y los de fuera de USA para ninguna de las métricas analizadas.

## T-test

In [ ]:
#Realizamos el t-test para las métricas seleccionadas
t_stat_ws, p_val_ws = stats.ttest_ind(us_players['WS'], out_us_players['WS'])
print(f'WS: t-statistic = {t_stat_ws}, p-value = {p_val_ws}')

t_stat_pts, p_val_pts = stats.ttest_ind(us_players['PTS per game'], out_us_players['PTS per game'])
print(f'PTS per game: t-statistic = {t_stat_pts}, p-value = {p_val_pts}')

t_stat_tbr, p_val_tbr = stats.ttest_ind(us_players['TBR per game'], out_us_players['TBR per game'])
print(f'TBR per game: t-statistic = {t_stat_tbr}, p-value = {p_val_tbr}')

t_stat_ast, p_val_ast = stats.ttest_ind(us_players['AST per game'], out_us_players['AST per game'])
print(f'AST per game: t-statistic = {t_stat_ast}, p-value = {p_val_ast}')

WS: t-statistic = -0.8529567500064283, p-value = 0.3938328197525107
PTS per game: t-statistic = -0.15787011770184084, p-value = 0.8745824782285969
TBR per game: t-statistic = -1.5884393745823173, p-value = 0.11241843501670501
AST per game: t-statistic = 0.7050802515745634, p-value = 0.4808804990373967


Dado que las varianzas son homogéneas (de acuerdo con la prueba de Levene), realizamos un t-test, ya que la suposición de igualdad de varianzas está satisfecha.

Para rechazar la hipótesis nula, el p-valor debe ser menor que el nivel de significancia (0.05). En este caso, dado que todos los p-valores son significativamente mayores que 0.05, no se puede rechazar la hipótesis nula para ninguna de las métricas de rendimiento analizadas. Con los datos disponibles, no hay evidencia suficiente para afirmar que hay una diferencia significativa en las métricas de rendimiento entre los jugadores de universidades americanas y los de fuera de USA.

## Verificación de la asimetría (o skewness) 

In [ ]:
#Realizamos la verificación de la asimetría

columns_of_interest = ['WS', 'PTS per game', 'TBR per game', 'AST per game']

for column in columns_of_interest:
    skew_us = skew(us_players[column])
    skew_out_us = skew(out_us_players[column])
    print(f'Asimetría para {column} en US players: {skew_us}')
    print(f'Asimetría para {column} en Out US players: {skew_out_us}')

Asimetría para WS en US players: 2.1232699939552426
Asimetría para WS en Out US players: 2.4403468108148556
Asimetría para PTS per game en US players: 0.7349067090060468
Asimetría para PTS per game en Out US players: 0.9006049152475369
Asimetría para TBR per game en US players: 1.6021301582056058
Asimetría para TBR per game en Out US players: 0.7628514778274799
Asimetría para AST per game en US players: 1.6330844402723215
Asimetría para AST per game en Out US players: 1.6590940705935933


La distribución de los datos presentan una distribución asimetría positiva, ya que el Skewness es mayor a 0. La asimetría positiva significativa en la mayoría de las métricas indica que los datos no presentan una distribución normal, por lo que realizamos una prueba de normalidad para verificar el resultado obtenido.

## Prueba de Normalidad (Shapiro-Wilk)

In [ ]:
#Realizamos la prueba de Mann-Whitney U para las métricas seleccionadas

columns_of_interest = ['WS', 'PTS per game', 'TBR per game', 'AST per game']

for column in columns_of_interest: 
    stat, p = shapiro(us_players[column])
    print(f'US players - {column}: Shapiro-Wilk stat = {stat}, p-value = {p}')
    
    stat, p = shapiro(out_us_players[column])
    print(f'Out US players - {column}: Shapiro-Wilk stat = {stat}, p-value = {p}')

US players - WS: Shapiro-Wilk stat = 0.7757511310355696, p-value = 1.6639869524045598e-38
Out US players - WS: Shapiro-Wilk stat = 0.6925471904380025, p-value = 4.2765131244585703e-13
US players - PTS per game: Shapiro-Wilk stat = 0.9575772191753695, p-value = 9.739657755810533e-19
Out US players - PTS per game: Shapiro-Wilk stat = 0.9310636498309208, p-value = 6.226288338723993e-05
US players - TBR per game: Shapiro-Wilk stat = 0.8978150014758322, p-value = 3.0356097776108345e-28
Out US players - TBR per game: Shapiro-Wilk stat = 0.9289249745140927, p-value = 4.715554558651426e-05
US players - AST per game: Shapiro-Wilk stat = 0.8516697558843919, p-value = 6.67971077088532e-33
Out US players - AST per game: Shapiro-Wilk stat = 0.8090945388585189, p-value = 5.457550890277046e-10


Relizando la prueba de Shapiro-Wilk, el p-valor es menor que el nivel de significancia (0.05). En este caso, todos los p-valores son extremadamente pequeños, lo que indica que los datos no siguen una distribución normal para todas las métricas.

## Prueba de Mann-Whitney U

In [ ]:
#Realizamos la prueba de Mann-Whitney U para las métricas seleccionadas

u_stat_ws, p_val_ws = mannwhitneyu(us_players['WS'], out_us_players['WS'])
print(f'WS: U-statistic = {u_stat_ws}, p-value = {p_val_ws}')

u_stat_pts, p_val_pts = mannwhitneyu(us_players['PTS per game'], out_us_players['PTS per game'])
print(f'PTS per game: U-statistic = {u_stat_pts}, p-value = {p_val_pts}')

u_stat_tbr, p_val_tbr = mannwhitneyu(us_players['TBR per game'], out_us_players['TBR per game'])
print(f'TBR per game: U-statistic = {u_stat_tbr}, p-value = {p_val_tbr}')

u_stat_ast, p_val_ast = mannwhitneyu(us_players['AST per game'], out_us_players['AST per game'])
print(f'AST per game: U-statistic = {u_stat_ast}, p-value = {p_val_ast}')

WS: U-statistic = 64084.0, p-value = 0.737670779608715
PTS per game: U-statistic = 63856.5, p-value = 0.7834331446128161
TBR per game: U-statistic = 56117.5, p-value = 0.07684130544848712
AST per game: U-statistic = 67898.5, p-value = 0.1792400653578473


Dado que nuestros datos no siguen una distribución normal y, las muestras no son pareadas, es decir, que son muestras independientes, realizamos la prueba de Mann-Whitney U. En base a los resultados de la prueba de Mann-Whitney U, el p-valor es mayor que el nivel de significancia (0.05), por lo que no encontramos evidencia suficiente para rechazar la hipótesis nula para ninguna de las métricas de rendimiento consideradas. 

Dado que los resultados de las pruebas estadísticas muestran que no hay diferencias significativas entre los dos grupos en ninguna de las métricas de rendimiento, no hay suficiente evidencia para rechazar la hipótesis nula (H0). Por lo tanto, se concluye que no hay diferencias significativas en el rendimiento entre los jugadores de que vienen de universidades americanas y el resto.

## Graficamos

In [ ]:
#Filtramos los grupos
us_players = top_20_pick[top_20_pick['College'] == 'US']
out_us_players = top_20_pick[top_20_pick['College'] == 'Out US']

#Combinamos los datos
combined_df = pd.concat([
    us_players.assign(College='US College'),
    out_us_players.assign(College='Non US College')])

#Renombramos las columnas
combined_df.rename(columns={
    'WS': 'Win Shares',
    'PTS per game': 'Points per Game',
    'TBR per game': 'Total Rebounds per Game',
    'AST per game': 'Assists per Game'}, inplace=True)

columns_of_interest = ['Win Shares', 'Points per Game', 'Total Rebounds per Game', 'Assists per Game']

#Creamos las gráficas
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=columns_of_interest)

for i, column in enumerate(columns_of_interest):
    row = i // 2 + 1
    col = i % 2 + 1
    violin_fig = px.violin(
        combined_df,
        y=column,
        x='College',
        color='College',
        box=True,
        points='all')
    
    for trace in violin_fig.data:
        trace.marker.color = 'royalblue' if trace.name == 'US College' else 'darkorange'
        fig.add_trace(trace, row=row, col=col)
    
    fig.update_xaxes(
        title=None, 
        showline=False, 
        showgrid=False,
        zeroline=False,
        showticklabels=False,
        row=row, col=col)
    fig.update_yaxes(
        title=None, 
        showline=False, 
        showgrid=False,
        zeroline=False,
        showticklabels=False,
        row=row, col=col)
    
fig.update_layout(
    title_text='Data Distribution and Comparison by College',
    paper_bgcolor='black',
    plot_bgcolor='black',
    font_color='white',
    legend_title=None,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01))

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1.1,
        xanchor="right",
        x=1.05))

fig.show()


NameError: name 'sp' is not defined